# *Student Performance & Behavior Analysis*

### Import the required liberaries 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-grading-dataset/Students_Grading_Dataset.csv
/kaggle/input/students-grading-dataset/metadata.xlsx
/kaggle/input/students-grading-dataset/Students_Grading_Dataset.json


# Load the dataset

In [2]:

df = pd.read_csv('/kaggle/input/students-grading-dataset/Students_Grading_Dataset.csv')

## Load the first 10 raws

In [3]:
df.head(10)

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1
5,S1005,Liam,Brown,student5@university.com,Male,21,Engineering,NaN,78.85,43.53,...,67.29,62.08,B,8.5,Yes,Yes,PhD,High,1,5.0
6,S1006,Ahmed,Jones,student6@university.com,Male,24,Business,57.60,66.26,89.07,...,93.65,83.21,F,21.3,No,Yes,NaN,Low,5,6.4
7,S1007,Ahmed,Smith,student7@university.com,Male,19,Engineering,51.91,45.67,73.96,...,93.24,81.93,F,27.3,Yes,No,NaN,Medium,4,4.3
8,S1008,Omar,Smith,student8@university.com,Female,21,CS,85.97,84.42,90.87,...,94.01,95.62,A,8.0,No,No,Bachelor's,Low,9,8.8
9,S1009,Sara,Smith,student9@university.com,Female,22,Engineering,64.01,87.96,98.47,...,78.60,84.99,A,9.6,No,Yes,NaN,Medium,10,6.4


## Dataset info

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Student_ID                  5000 non-null   object 
 1   First_Name                  5000 non-null   object 
 2   Last_Name                   5000 non-null   object 
 3   Email                       5000 non-null   object 
 4   Gender                      5000 non-null   object 
 5   Age                         5000 non-null   int64  
 6   Department                  5000 non-null   object 
 7   Attendance (%)              4484 non-null   float64
 8   Midterm_Score               5000 non-null   float64
 9   Final_Score                 5000 non-null   float64
 10  Assignments_Avg             4483 non-null   float64
 11  Quizzes_Avg                 5000 non-null   float64
 12  Participation_Score         5000 non-null   float64
 13  Projects_Score              5000 

## Classify columns as categorical or numerical

In [5]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()


print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)


Categorical Columns: ['Student_ID', 'First_Name', 'Last_Name', 'Email', 'Gender', 'Department', 'Grade', 'Extracurricular_Activities', 'Internet_Access_at_Home', 'Parent_Education_Level', 'Family_Income_Level']
Numerical Columns: ['Age', 'Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score', 'Projects_Score', 'Total_Score', 'Study_Hours_per_Week', 'Stress_Level (1-10)', 'Sleep_Hours_per_Night']


## Check duplicates 

In [6]:
print(df['Student_ID'].duplicated().sum())

0


## Find the null values and their percentage 

In [7]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Filter only columns with missing values
missing_summary = pd.DataFrame({
    'Missing Values': missing_values[missing_values > 0],
    'Percentage (%)': missing_percentage[missing_values > 0]
}).sort_values(by='Percentage (%)', ascending=False)

# Display the summary
print(missing_summary)

                        Missing Values  Percentage (%)
Parent_Education_Level            1794           35.88
Assignments_Avg                    517           10.34
Attendance (%)                     516           10.32


## Check if there negative values

In [8]:
columns_to_check = ['Age', 'Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg',
                   'Quizzes_Avg','Participation_Score','Total_Score','Study_Hours_per_Week',
                   'Stress_Level (1-10)', 'Sleep_Hours_per_Night']  
negative_values = df[(df[columns_to_check] < 0).any(axis=1)]
print(negative_values.sum())

Student_ID                      0
First_Name                      0
Last_Name                       0
Email                           0
Gender                          0
Age                             0
Department                      0
Attendance (%)                0.0
Midterm_Score                 0.0
Final_Score                   0.0
Assignments_Avg               0.0
Quizzes_Avg                   0.0
Participation_Score           0.0
Projects_Score                0.0
Total_Score                   0.0
Grade                           0
Study_Hours_per_Week          0.0
Extracurricular_Activities      0
Internet_Access_at_Home         0
Parent_Education_Level          0
Family_Income_Level             0
Stress_Level (1-10)             0
Sleep_Hours_per_Night         0.0
dtype: object


## Fill missing values

In [9]:

df['Attendance (%)'].fillna(df['Attendance (%)'].mean(), inplace=True)
df['Assignments_Avg'].fillna(df['Assignments_Avg'].mean(), inplace=True)
df['Parent_Education_Level'].fillna('Unknown', inplace=True)


<ipython-input-9-73f5b765ace9>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance (%)'].fillna(df['Attendance (%)'].mean(), inplace=True)
<ipython-input-9-73f5b765ace9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

## Unique values extraction for analysis

In [10]:
unique_values = {col: df[col].unique() for col in df.columns}
for col, values in unique_values.items():
    print(f"{col}: {values}\n")

Student_ID: ['S1000' 'S1001' 'S1002' ... 'S5997' 'S5998' 'S5999']

First_Name: ['Omar' 'Maria' 'Ahmed' 'John' 'Liam' 'Sara' 'Emma' 'Ali']

Last_Name: ['Williams' 'Brown' 'Jones' 'Smith' 'Davis' 'Johnson']

Email: ['student0@university.com' 'student1@university.com'
 'student2@university.com' ... 'student4997@university.com'
 'student4998@university.com' 'student4999@university.com']

Gender: ['Female' 'Male']

Age: [22 18 24 23 21 19 20]

Department: ['Engineering' 'Business' 'Mathematics' 'CS']

Attendance (%): [52.29 97.27 57.19 ... 55.54 99.21 83.92]

Midterm_Score: [55.03 97.23 67.05 ... 69.92 86.31 83.24]

Final_Score: [57.82 45.8  93.68 ... 55.48 49.8  64.21]

Assignments_Avg: [84.22       74.79867276 67.7        ... 59.74       75.26
 81.2       ]

Quizzes_Avg: [74.06 94.24 85.7  ... 56.92 99.32 83.51]

Participation_Score: [ 3.99  8.32  5.05  6.54  5.97  6.38  2.3   3.73  0.51  5.88  4.59  7.61
  2.06  4.37  1.81  6.14  8.48  7.65  9.44  9.72  7.72  2.87  2.19  4.48
  9.94  4.1

*** # According to exectraction, the dataset is for 8 students with 5000 unique students IDs > 'S1000' to 'S5999' for the very same students !

- First_Name: ['Omar' 'Maria' 'Ahmed' 'John' 'Liam' 'Sara' 'Emma' 'Ali']
- Last_Name: ['Williams' 'Brown' 'Jones' 'Smith' 'Davis' 'Johnson']

## Top 10 students 


In [11]:
top_students = df[['Student_ID', 'First_Name', 'Last_Name', 'Total_Score', 'Grade', 'Department',
                  'Age', 'Gender']].sort_values(by='Total_Score', ascending=False).head(10)

print(top_students)

     Student_ID First_Name Last_Name  Total_Score Grade   Department  Age  \
330       S1330        Ali     Brown        99.99     B  Engineering   22   
911       S1911      Ahmed  Williams        99.99     A           CS   20   
2140      S3140      Maria   Johnson        99.99     B           CS   23   
1819      S2819       John     Davis        99.98     F     Business   20   
2632      S3632        Ali   Johnson        99.98     C           CS   23   
152       S1152        Ali     Davis        99.96     C     Business   23   
4982      S5982        Ali  Williams        99.95     C  Mathematics   24   
1612      S2612       Liam   Johnson        99.94     D  Mathematics   21   
378       S1378       Sara     Smith        99.91     D  Mathematics   24   
487       S1487      Maria   Johnson        99.90     C           CS   20   

      Gender  
330     Male  
911   Female  
2140    Male  
1819  Female  
2632  Female  
152     Male  
4982    Male  
1612    Male  
378   Female  
48

## Drop unnessesary columns 


In [12]:
df = df.drop(columns=['Student_ID', 'First_Name', 'Last_Name', 'Email'])

## Count students according to their ages and group by Age

In [13]:
age_distribution = df.groupby(['Age']).size().reset_index(name='Count')
print(age_distribution)


   Age  Count
0   18    682
1   19    705
2   20    671
3   21    753
4   22    732
5   23    734
6   24    723


## Distribution of Students' Ages"

In [14]:
plt.figure(figsize=(8, 5))
sns.countplot(x=df['Age'], palette='viridis')

plt.title('Distribution of Students Ages', fontsize=14)
plt.xlabel('Student Age', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

NameError: name 'plt' is not defined

## categorize students according to their study programs - Age 

- Diploma students: 1-2 years
- Associate Degree students: 2 years
- Bachelor’s Degree students : 4 years
- Master’s Degree students : 6 years


In [ ]:
bins = [17, 19, 21, 23, 25]  
labels = ['Diploma', 'Associate Degree', 'Bachelor Degree', 'Master’s Degree']  
df['Study_Program'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
print(df[['Age', 'Study_Program']].head(10))


## Count number of students in each Study Program by Gender

In [ ]:
age_group_counts = df.groupby(['Study_Program', 'Gender']).size().unstack()
# Plot grouped bar chart
age_group_counts.plot(kind='bar', figsize=(10, 6), colormap='coolwarm', edgecolor='black')
plt.title('Student Distribution by Study Program and Gender')
plt.xlabel('Study Program')
plt.ylabel('Number of Students')
plt.xticks(rotation=15)
plt.legend(title='Gender')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()


## Count the number of students in each age group, split by gender


In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x='Age', hue='Gender', data=df, palette='Set2')

# Customize the plot
plt.xlabel('Age')
plt.ylabel('Number of Students')
plt.title('Distribution of Students by Age and Gender')
plt.legend(title='Gender')
plt.show()


## Function to detect outliers per feature using IQR

In [ ]:
def detect_outliers(df):
    outlier_counts = {}

    for col in df.select_dtypes(include=['number']).columns:  # Only numerical columns
        Q1 = df[col].quantile(0.25)  # 25th percentile
        Q3 = df[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col].count()
        
        if outlier_count > 0:  # Only store columns that have outliers
            outlier_counts[col] = outlier_count

    return outlier_counts

# Apply function and display outliers
outliers_summary = detect_outliers(df)
print('Outlier Counts per feature:', outliers_summary)

## Export the grades

In [ ]:
df['Grade'].unique()

## Count of students per grade, grouped by Gender

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.countplot(data=df, x='Grade', hue='Gender', order=['A', 'B', 'C', 'D', 'F'], palette='winter')

plt.xlabel('Grade')
plt.ylabel('Number of Students')
plt.title('Distribution of Grades by Gender (Ordered)')
plt.legend(title='Gender')

# Label bars with exact count values
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge')

plt.show()


## Count of students per grade, grouped by Study Year

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='Grade', hue='Study_Program', order=['A', 'B', 'C', 'D', 'F'], palette='Set3')
plt.xlabel('Grade')
plt.ylabel('Number of Students')
plt.title('Distribution of Grades by Study Program')
plt.legend(title='Study Program')
plt.show()


## Distribution of Grades by Gender (Ordered)

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.countplot(data=df, x='Grade', hue='Gender', order=['A', 'B', 'C', 'D', 'F'], palette='Set1')

# Annotate each bar with the count
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=12, color='black', fontweight='bold')

plt.xlabel('Grade')
plt.ylabel('Number of Students')
plt.title('Distribution of Grades by Gender (Ordered)')
plt.legend(title='Gender')
plt.show()

## Count the number of students in each grade

In [ ]:
import plotly.express as px
grade_counts = df['Grade'].value_counts().reset_index()
grade_counts.columns = ['Grade', 'Number of Students']  # Rename columns for clarity

# Create a bar chart
fig_grades = px.bar(
    grade_counts, 
    x='Grade', 
    y='Number of Students', 
    text_auto=True,  # Show values on bars
    height=600,  
    width=1000,
    color_discrete_sequence=['royalblue']  # Use a single color for clarity
)

# Improve readability
fig_grades.update_layout(
    yaxis=dict(title='Number of Students'),  
    xaxis=dict(title='Grade', tickangle=-30),  
    bargap=0.2,  
    title="Student Grade Distribution"
)

fig_grades.show()

## # Correlotion 1
- Study Hours vs. Final Score

In [ ]:
# Correlotion 1
# Study Hours vs. Final Score
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x='Study_Hours_per_Week', y='Final_Score', alpha=0.6)
plt.title('Study Hours vs. Final Score')
plt.xlabel('Study Hours per Week')
plt.ylabel('Final Score')
plt.show()

- There's a moderate positive correlation between Study_Hours_per_Week and Final_Score. 
- This means that students who study more tend to perform better.

## #Correlotion 2
- Parental Education vs. Final Score

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='Parent_Education_Level', y='Final_Score', palette='coolwarm')
plt.xticks(rotation=45)
plt.title('Parental Education Level vs. Final Score')
plt.xlabel('Parent Education Level')
plt.ylabel('Final Score')
plt.show()

-  Weak correlation. The parental education level does not show a strong direct impact on Final_Score

## # Correlation 3
- Stress Level vs. Final Score

In [ ]:
plt.figure(figsize=(8, 5))
sns.regplot(data=df, x='Stress_Level (1-10)', y='Final_Score', scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title('Stress Level vs. Final Score')
plt.xlabel('Stress Level (1-10)')
plt.ylabel('Final Score')
plt.show()

-There's a negative correlation between Stress_Level (1-10) and Final_Score, meaning higher stress levels generally lead to lower scores.

## #Correlation 4
- Department vs. Average Final Score

In [ ]:
plt.figure(figsize=(10, 5))
dept_avg_score = df.groupby('Department')['Final_Score'].mean().reset_index()
sns.barplot(data=dept_avg_score, x='Department', y='Final_Score', palette='viridis')
plt.xticks(rotation=30)
plt.title('Department vs. Average Final Score')
plt.xlabel('Department')
plt.ylabel('Average Final Score')
plt.show()

In [ ]:
# Confirm null values are filled 

# Calculate missing values count and percentage
missing_values = df.isnull().sum()

# Display the summary
print(missing_values)

In [ ]:
# Function to detect outliers using IQR
def detect_outliers(df):
    outlier_counts = {}

    for col in df.select_dtypes(include=['number']).columns:  # Only numerical columns
        Q1 = df[col].quantile(0.25)  # 25th percentile
        Q3 = df[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col].count()
        
        if outlier_count > 0:  # Only store columns that have outliers
            outlier_counts[col] = outlier_count

    return outlier_counts

# Apply function and display outliers
outliers_summary = detect_outliers(df)
print("Outlier Counts per Column:", outliers_summary)


In [ ]:
# Select numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns

# Set figure size
plt.figure(figsize=(15, len(numerical_cols) * 4))

# Create a subplot for each numerical column
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(len(numerical_cols), 1, i)  # Create separate plots
    sns.boxplot(x=df[col], color='skyblue')
    plt.title(f"Boxplot of {col}")
    plt.xlabel("")  # Remove x-label for cleaner visualization
    plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

